In [ ]:
!lscpu

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   8
  On-line CPU(s) list:    0-7
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:           6
    Model:                79
    Thread(s) per core:   2
    Core(s) per socket:   4
    Socket(s):            1
    Stepping:             0
    BogoMIPS:             4399.99
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 cl
                          flush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc re
                          p_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3
                           fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand
                           hypervisor lahf_lm abm 3dnowprefetch i

**Load The Data**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import time
from collections import deque
from tqdm import tqdm
from typing import Dict, List, Tuple
import math

In [4]:
import random
import numpy as np
from numpy import ndarray

In [ ]:
# Set seeds for reproducibility
# --- Fixed parameters ---
dim = 7 # dimension of the points
np.random.seed(42)  # set seed for reproducibility

In [ ]:
# Define the dataset path
dataset_path = '/content/drive/MyDrive/datasets/dataSearch/higgs_ds_mod_orig.txt'

# Load the dataset
# If the file has a different delimiter (e.g., commas), you can set delimiter=',' accordingly.
dataset = np.loadtxt(dataset_path)
total_points = len(dataset)

In [ ]:
# Data sizes:
SB = 10_000    # Small Batch: 10K points
MB = 100_000   # Medium Batch: 100K points
LB = 1_000_000 # Large Batch: 1M points

# Query sizes:
SQ = 20        # Small Query: 20 queries
MQ = 100       # Medium Query: 100 queries
LQ = 500       # Large Query: 500 queries

# === Define the test combinations ===
# Each tuple is of the form (data_size, query_size)
test_combinations = [
    (LB, MQ)
]

In [ ]:
# ----------------------------
# Phase 1: Pre-generate and Store Queries from the Dataset
# ----------------------------

queries_list = []
st_idx = 0  # Starting index for slicing the dataset

for data_size, query_size in test_combinations:
    # Slice the current data chunk
    end_idx = st_idx + data_size
    current_data = dataset[st_idx:end_idx]
    st_idx = end_idx  # Update start index for next chunk

    # Ensure query selection is different for each batch size
    if query_size <= len(current_data):  # Sampling from the current batch only
        random_indices = np.random.choice(len(current_data), size=query_size, replace=False)
        queries = current_data[random_indices]  # Select query points from the current batch
    else:
        raise ValueError(f"Query size {query_size} exceeds available data points {len(current_data)}.")

    # Append the selected queries to the list
    queries_list.append(queries)


**-----------------Ball Tree--------------------------**

In [ ]:
import numpy as np
from collections import deque

#*-----------------------------------------------------------------------------
class BallNode:
    def __init__(self):
        self.center = None
        self.radius = None
        self.left = None
        self.right = None
        self.points = None
        self.pivot1 = None
        self.pivot2 = None

#*-----------------------------------------------------------------------------
class BallTree:
    def __init__(self, leaf_size=220):
        self.leaf_size = leaf_size
        self.root = None

    #*-----------------------------------------------------------------------------
    def _find_furthest_points(self, points):
        # First find point furthest from origin as first pivot
        distances = np.sum(points ** 2, axis=1)
        p1_idx = np.argmax(distances)
        pivot1 = points[p1_idx]

        # Find point furthest from first pivot
        distances = np.sum((points - pivot1) ** 2, axis=1)
        p2_idx = np.argmax(distances)
        pivot2 = points[p2_idx]

        return pivot1, pivot2

    #*-----------------------------------------------------------------------------
    def _compute_distances(self, points, pivot):
        return np.sqrt(np.sum((points - pivot) ** 2, axis=1))

    #*-----------------------------------------------------------------------------
    def build(self, points):
        if points is None or len(points) == 0:
            return None

        points = np.array(points)
        self.root = self._build_recursive(points)
        return self.root

    #*-----------------------------------------------------------------------------
    def _build_recursive(self, points):
        if len(points) <= self.leaf_size:
            node = BallNode()
            node.points = points
            node.center = np.mean(points, axis=0)
            if len(points) > 1:
                node.radius = np.max(self._compute_distances(points, node.center))
            else:
                node.radius = 0
            return node

        node = BallNode()

        # Find two furthest points as pivots
        pivot1, pivot2 = self._find_furthest_points(points)
        node.pivot1 = pivot1
        node.pivot2 = pivot2

        # Assign points to closest pivot
        dist_to_p1 = self._compute_distances(points, pivot1)
        dist_to_p2 = self._compute_distances(points, pivot2)

        left_mask = dist_to_p1 <= dist_to_p2
        left_points = points[left_mask]
        right_points = points[~left_mask]

        # Compute node properties
        node.center = np.mean(points, axis=0)
        node.radius = np.max(self._compute_distances(points, node.center))

        # Recursively build subtrees
        node.left = self._build_recursive(left_points)
        node.right = self._build_recursive(right_points)

        return node

    #*-----------------------------------------------------------------------------
    def search(self, point):
        if self.root is None:
            return float('inf'), None

        point = np.array(point)
        best_dist = float('inf')
        best_point = None

        # Use a priority queue for best-first search
        # Each entry is (lower_bound_distance, node)
        queue = deque([(0, self.root)])

        while queue:
            _, node = queue.popleft()  # Get the node with lowest lower bound distance

            # Calculate distance from query point to node center
            dist_to_center = np.linalg.norm(point - node.center)

            # If the closest possible point in this ball is farther than our best,
            # we can skip this subtree
            if dist_to_center - node.radius > best_dist:
                continue

            if node.points is not None:
                # Leaf node - check all points
                distances = np.linalg.norm(node.points - point, axis=1)
                min_idx = np.argmin(distances)
                min_dist = distances[min_idx]

                if min_dist < best_dist:
                    best_dist = min_dist
                    best_point = node.points[min_idx]
            else:
                # Internal node - decide which child to visit first
                dist_to_p1 = np.linalg.norm(point - node.pivot1)
                dist_to_p2 = np.linalg.norm(point - node.pivot2)

                # Calculate lower bound distances to both subtrees
                # and visit them in order of potential closeness
                if dist_to_p1 <= dist_to_p2:
                    # Visit left subtree first, then right
                    if node.left:
                        left_lb = max(0, dist_to_center - node.left.radius)
                        queue.appendleft((left_lb, node.left))  # Add to front to visit next

                    if node.right:
                        right_lb = max(0, dist_to_center - node.right.radius)
                        if right_lb <= best_dist:  # Only add if it could contain a closer point
                            queue.append((right_lb, node.right))
                else:
                    # Visit right subtree first, then left
                    if node.right:
                        right_lb = max(0, dist_to_center - node.right.radius)
                        queue.appendleft((right_lb, node.right))  # Add to front to visit next

                    if node.left:
                        left_lb = max(0, dist_to_center - node.left.radius)
                        if left_lb <= best_dist:  # Only add if it could contain a closer point
                            queue.append((left_lb, node.left))

        return best_dist, best_point
#*-----------------------------------------------------------------------------

**Ball Tree Large Scale Examination**

In [ ]:
import time
from tqdm import tqdm

# Test code for Ball Tree
results = []

# Define leaf size range
leaf_size_range_start = 50
leaf_size_range_end = 10_000  # Ensure inclusive range
leaf_size_step = 50

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, leaf_size in enumerate(range(leaf_size_range_start, leaf_size_range_end + leaf_size_step, leaf_size_step)):
    print(f"\n{idx+1}: === Testing with Leaf Size: {leaf_size:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the Ball Tree ---
    build_start = time.perf_counter()
    btree = BallTree(leaf_size=leaf_size)
    btree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc=f"Querying (Leaf Size {leaf_size:,})"):
        search_start = time.perf_counter()
        r_min, nn_pt = btree.search(query)
        search_time += time.perf_counter() - search_start

    total_time = build_time + search_time
    results.append((leaf_size, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Leaf Size: {leaf_size:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of Ball Tree Node Size Test ===")
for l_size, b_time, s_time, t_time in results:
    print(f"Leaf Size: {l_size}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Leaf Size: 50, Data Size: 1,000,000 points, Query Size: 1,000 queries ===


Querying (Leaf Size 50): 100%|██████████| 1000/1000 [00:08<00:00, 122.30it/s]


-------------------------------------------------------------
Build Time: 7.3622 sec, Search Time: 8.0974 sec
Leaf Size: 50, Total Time: 15.4596 sec

2: === Testing with Leaf Size: 100, Data Size: 1,000,000 points, Query Size: 1,000 queries ===


Querying (Leaf Size 100): 100%|██████████| 1000/1000 [00:07<00:00, 128.00it/s]


-------------------------------------------------------------
Build Time: 5.5961 sec, Search Time: 7.7395 sec
Leaf Size: 100, Total Time: 13.3356 sec

3: === Testing with Leaf Size: 150, Data Size: 1,000,000 points, Query Size: 1,000 queries ===


...

-------------------------------------------------------------
Build Time: 2.5128 sec, Search Time: 17.4657 sec
Leaf Size: 9,900, Total Time: 19.9785 sec

199: === Testing with Leaf Size: 9,950, Data Size: 1,000,000 points, Query Size: 1,000 queries ===


Querying (Leaf Size 9,950): 100%|██████████| 1000/1000 [00:17<00:00, 55.76it/s]


-------------------------------------------------------------
Build Time: 2.6036 sec, Search Time: 17.7703 sec
Leaf Size: 9,950, Total Time: 20.3739 sec

200: === Testing with Leaf Size: 10,000, Data Size: 1,000,000 points, Query Size: 1,000 queries ===


Querying (Leaf Size 10,000): 100%|██████████| 1000/1000 [00:18<00:00, 54.98it/s]

-------------------------------------------------------------
Build Time: 2.6669 sec, Search Time: 18.0178 sec
Leaf Size: 10,000, Total Time: 20.6847 sec

=== Summary of Ball Tree Node Size Test ===
Leaf Size: 50, Build Time: 7.3622 sec, Search Time: 8.0974 sec, Total Time: 15.4596 sec
Leaf Size: 100, Build Time: 5.5961 sec, Search Time: 7.7395 sec, Total Time: 13.3356 sec
Leaf Size: 150, Build Time: 4.9743 sec, Search Time: 7.5807 sec, Total Time: 12.5550 sec
Leaf Size: 200, Build Time: 4.7135 sec, Search Time: 7.4125 sec, Total Time: 12.1259 sec
Leaf Size: 250, Build Time: 4.5950 sec, Search Time: 7.2498 sec, Total Time: 11.8448 sec
Leaf Size: 300, Build Time: 4.4151 sec, Search Time: 7.2097 sec, Total Time: 11.6249 sec
Leaf Size: 350, Build Time: 4.3009 sec, Search Time: 7.2031 sec, Total Time: 11.5040 sec
Leaf Size: 400, Build Time: 4.2253 sec, Search Time: 7.2478 sec, Total Time: 11.4731 sec
Leaf Size: 450, Build Time: 4.0607 sec, Search Time: 7.0801 sec, Total Time: 11.1409 sec
L

**Ball Tree Large Scale (Run 2)**

In [ ]:
import time
from tqdm import tqdm

# Test code for Ball Tree
results = []

# Define leaf size range
leaf_size_range_start = 50
leaf_size_range_end = 10_000  # Ensure inclusive range
leaf_size_step = 50

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, leaf_size in enumerate(range(leaf_size_range_start, leaf_size_range_end + leaf_size_step, leaf_size_step)):
    print(f"\n{idx+1}: === Testing with Leaf Size: {leaf_size:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the Ball Tree ---
    build_start = time.perf_counter()
    btree = BallTree(leaf_size=leaf_size)
    btree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc=f"Querying (Leaf Size {leaf_size:,})"):
        search_start = time.perf_counter()
        r_min, nn_pt = btree.search(query)
        search_time += time.perf_counter() - search_start

    total_time = build_time + search_time
    results.append((leaf_size, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Leaf Size: {leaf_size:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of Ball Tree Node Size Test ===")
for l_size, b_time, s_time, t_time in results:
    print(f"Leaf Size: {l_size}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Leaf Size: 50, Data Size: 1,000,000 points, Query Size: 1,000 queries ===


Querying (Leaf Size 50): 100%|██████████| 1000/1000 [00:08<00:00, 121.63it/s]


-------------------------------------------------------------
Build Time: 7.0142 sec, Search Time: 8.1423 sec
Leaf Size: 50, Total Time: 15.1565 sec

2: === Testing with Leaf Size: 100, Data Size: 1,000,000 points, Query Size: 1,000 queries ===


Querying (Leaf Size 100): 100%|██████████| 1000/1000 [00:07<00:00, 126.08it/s]


-------------------------------------------------------------
Build Time: 5.6414 sec, Search Time: 7.8563 sec
Leaf Size: 100, Total Time: 13.4977 sec

3: === Testing with Leaf Size: 150, Data Size: 1,000,000 points, Query Size: 1,000 queries ===


...

-------------------------------------------------------------
Build Time: 2.5513 sec, Search Time: 17.4948 sec
Leaf Size: 9,900, Total Time: 20.0461 sec

199: === Testing with Leaf Size: 9,950, Data Size: 1,000,000 points, Query Size: 1,000 queries ===


Querying (Leaf Size 9,950): 100%|██████████| 1000/1000 [00:17<00:00, 56.18it/s]


-------------------------------------------------------------
Build Time: 2.5569 sec, Search Time: 17.6417 sec
Leaf Size: 9,950, Total Time: 20.1986 sec

200: === Testing with Leaf Size: 10,000, Data Size: 1,000,000 points, Query Size: 1,000 queries ===


Querying (Leaf Size 10,000): 100%|██████████| 1000/1000 [00:17<00:00, 56.31it/s]

-------------------------------------------------------------
Build Time: 2.5415 sec, Search Time: 17.6048 sec
Leaf Size: 10,000, Total Time: 20.1463 sec

=== Summary of Ball Tree Node Size Test ===
Leaf Size: 50, Build Time: 7.0142 sec, Search Time: 8.1423 sec, Total Time: 15.1565 sec
Leaf Size: 100, Build Time: 5.6414 sec, Search Time: 7.8563 sec, Total Time: 13.4977 sec
Leaf Size: 150, Build Time: 5.0208 sec, Search Time: 7.5471 sec, Total Time: 12.5680 sec
Leaf Size: 200, Build Time: 4.7196 sec, Search Time: 7.3902 sec, Total Time: 12.1099 sec
Leaf Size: 250, Build Time: 4.5032 sec, Search Time: 7.2619 sec, Total Time: 11.7652 sec
Leaf Size: 300, Build Time: 4.3800 sec, Search Time: 7.1546 sec, Total Time: 11.5346 sec
Leaf Size: 350, Build Time: 4.1634 sec, Search Time: 7.1466 sec, Total Time: 11.3100 sec
Leaf Size: 400, Build Time: 4.0799 sec, Search Time: 7.2408 sec, Total Time: 11.3206 sec
Leaf Size: 450, Build Time: 4.1119 sec, Search Time: 7.1695 sec, Total Time: 11.2814 sec
L

**-------Optimum Node Size for Rebuild---------**

In [ ]:
import numpy as np

# Leaf sizes
leaf_sizes = np.array([   50,  100,  150,  200,  250,  300,  350,  400,  450,  500,  550,  600,
   650,  700,  750,  800,  850,  900,  950, 1000, 1050, 1100, 1150, 1200,
  1250, 1300, 1350, 1400, 1450, 1500, 1550, 1600, 1650, 1700, 1750, 1800,
  1850, 1900, 1950, 2000, 2050, 2100, 2150, 2200, 2250, 2300, 2350, 2400,
  2450, 2500, 2550, 2600, 2650, 2700, 2750, 2800, 2850, 2900, 2950, 3000,
  3050, 3100, 3150, 3200, 3250, 3300, 3350, 3400, 3450, 3500, 3550, 3600,
  3650, 3700, 3750, 3800, 3850, 3900, 3950, 4000, 4050, 4100, 4150, 4200,
  4250, 4300, 4350, 4400, 4450, 4500, 4550, 4600, 4650, 4700, 4750, 4800,
  4850, 4900, 4950, 5000, 5050, 5100, 5150, 5200, 5250, 5300, 5350, 5400,
  5450, 5500, 5550, 5600, 5650, 5700, 5750, 5800, 5850, 5900, 5950, 6000,
  6050, 6100, 6150, 6200, 6250, 6300, 6350, 6400, 6450, 6500, 6550, 6600,
  6650, 6700, 6750, 6800, 6850, 6900, 6950, 7000, 7050, 7100, 7150, 7200,
  7250, 7300, 7350, 7400, 7450, 7500, 7550, 7600, 7650, 7700, 7750, 7800,
  7850, 7900, 7950, 8000, 8050, 8100, 8150, 8200, 8250, 8300, 8350, 8400,
  8450, 8500, 8550, 8600, 8650, 8700, 8750, 8800, 8850, 8900, 8950, 9000,
  9050, 9100, 9150, 9200, 9250, 9300, 9350, 9400, 9450, 9500, 9550, 9600,
  9650, 9700, 9750, 9800, 9850, 9900, 9950,10000])

# Total times from two runs (or versions)
total_times_1 = np.array([15.4596,13.3356,12.555 ,12.1259,11.8448,11.6249,11.504 ,11.4731,11.1409,
 11.0562,10.9141,10.9728,10.7493,10.8064,10.9096,10.7608,10.821 ,10.9066,
 10.8421,10.9519,10.727 ,10.7751,10.9005,10.8467,10.9531,10.9942,10.9847,
 11.0477,11.0062,11.0647,11.091 ,11.2097,11.2736,11.1882,11.3121,11.5042,
 11.398 ,11.4797,11.4373,11.5645,11.5796,11.6374,11.6824,11.664 ,11.7722,
 11.9102,11.8514,11.8684,11.8495,12.003 ,12.0611,12.1616,12.2077,12.4519,
 12.5202,12.7116,12.6023,12.5733,12.6506,12.7768,12.976 ,13.0377,12.9211,
 13.0011,13.1652,13.0786,12.9493,13.0923,13.1666,13.4301,13.3845,13.3697,
 13.3107,13.3534,13.4782,13.5285,13.5759,13.7621,13.9039,13.9647,14.1378,
 14.0801,14.0621,14.1736,14.3098,14.1205,14.0447,14.2048,14.1721,14.2113,
 14.438 ,14.5509,14.685 ,14.7914,14.7817,14.7839,14.8499,14.6664,14.7319,
 14.9106,14.9014,14.8751,14.915 ,14.9105,14.892 ,14.8065,14.91  ,14.8363,
 14.8973,14.8711,14.8789,15.0431,14.9941,15.3404,15.2253,15.2439,15.316 ,
 15.2907,15.3915,15.5058,15.5982,15.6759,15.7355,15.643 ,15.8837,15.8725,
 16.1848,16.0712,16.2291,16.0716,16.245 ,16.2634,16.4259,16.2481,16.2178,
 16.1247,16.2168,16.2197,16.2951,16.3979,16.5385,16.5301,16.3245,16.5863,
 16.54  ,16.6233,16.4967,16.7095,16.6509,16.8499,16.8853,17.052 ,16.9387,
 17.0256,16.9749,17.1822,17.0451,17.0968,16.9688,17.1108,17.089 ,17.2825,
 17.8218,17.7012,17.8586,18.0045,17.9858,18.1674,18.5367,18.8011,18.9867,
 19.0229,19.189 ,19.2014,19.4292,19.5228,19.3589,19.3866,19.4282,19.4662,
 19.7624,19.7401,19.3188,19.4103,19.7372,19.6788,19.5007,19.8497,19.6075,
 19.6416,19.8405,20.2705,19.9645,19.9732,19.7541,19.9109,19.9972,19.9785,
 20.3739,20.6847])

total_times_2 = np.array([15.1565,13.4977,12.568 ,12.1099,11.7652,11.5346,11.31  ,11.3206,11.2814,
 11.1035,11.1779,11.087 ,10.9497,10.9848,10.7026,10.7252,10.8049,10.7987,
 10.8799,10.8924,10.9019,10.7947,10.8695,11.0001,11.192 ,11.1314,11.1231,
 10.9465,11.1579,11.2277,11.2365,11.3368,11.3005,11.4818,11.4938,11.4825,
 11.5247,11.5726,11.6728,11.5591,11.7456,11.671 ,11.7979,11.7538,11.8359,
 12.0029,12.0022,12.1   ,12.192 ,11.9672,12.2695,12.1624,12.385 ,12.4459,
 12.3884,12.4131,12.565 ,12.6748,12.768 ,12.5523,12.5764,13.0039,12.9296,
 12.9271,13.0661,13.1222,12.9633,13.1836,13.5133,13.395 ,13.6447,13.4182,
 13.408 ,13.4773,13.5143,13.6851,13.8514,13.8887,14.0648,14.1292,14.2845,
 14.2599,14.1931,14.113 ,14.2665,14.3383,14.4196,14.5379,14.489 ,14.5398,
 14.6749,14.7316,14.7227,14.8395,14.9654,15.071 ,15.1413,14.9698,15.2051,
 15.2752,15.5548,15.2397,15.2315,15.7456,15.6986,15.9345,15.7714,15.4781,
 15.5339,15.6178,15.6685,15.7325,15.6309,16.0493,15.894 ,15.9296,15.8824,
 16.0458,16.124 ,16.3125,16.7272,16.398 ,16.4016,16.5372,16.9944,17.1094,
 17.3992,17.3825,17.3772,17.3489,17.155 ,17.1771,17.2362,17.2147,17.2492,
 17.0858,17.0876,17.2445,17.254 ,17.3567,17.443 ,17.5755,17.5953,17.6597,
 17.7292,17.7179,17.5216,17.9352,17.8548,17.8908,17.7359,18.2043,18.1709,
 18.5497,18.8434,18.8872,18.7253,18.4798,18.9386,18.6774,18.7884,18.9112,
 18.5868,19.3581,19.1868,18.9181,18.7173,18.9041,18.8309,19.0395,18.9974,
 18.8488,19.1895,19.1424,18.9972,19.3333,19.6033,19.4844,19.5706,19.5814,
 19.626 ,19.6657,19.5523,19.7035,19.6553,19.965 ,19.5653,19.6091,19.717 ,
 19.8491,20.2206,20.4885,20.567 ,19.9931,19.9359,19.9615,20.0202,20.0461,
 20.1986,20.1463])

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_leaf_size = leaf_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at leaf size: {optimal_leaf_size}")


Average total time (sec): 10.7430
Minimum found at leaf size: 800


In [6]:
build_time_1 = np.array([7.3622,5.5961,4.9743,4.7135,4.595 ,4.4151,4.3009,4.2253,4.0607,4.0176,
 3.9437,3.8595,3.7981,3.766 ,3.7404,3.6989,3.685 ,3.621 ,3.6936,3.6038,
 3.5978,3.5055,3.5062,3.4623,3.4717,3.4655,3.4246,3.4308,3.4193,3.388 ,
 3.3506,3.3785,3.345 ,3.351 ,3.3347,3.3839,3.3287,3.2645,3.2634,3.2406,
 3.2653,3.2526,3.1986,3.1964,3.1725,3.1976,3.1337,3.1179,3.1027,3.1082,
 3.1224,3.072 ,3.0898,3.0973,3.1251,3.1143,3.1041,3.0757,3.0708,3.1169,
 3.0955,3.143 ,3.0592,3.0516,3.0777,3.0325,3.0033,2.9816,3.0051,3.0301,
 3.0644,2.9581,2.9452,2.9431,2.9285,2.9412,2.97  ,2.9352,2.913 ,2.9295,
 3.0347,2.9419,2.9084,2.9317,3.0011,2.8994,2.8377,2.878 ,2.8189,2.7713,
 2.8141,2.8418,2.8742,2.8666,2.838 ,2.8067,2.808 ,2.8034,2.8305,2.7662,
 2.7629,2.7908,2.8158,2.7699,2.7524,2.7518,2.7463,2.7079,2.7414,2.735 ,
 2.7087,2.7331,2.6825,2.7675,2.7224,2.6634,2.6831,2.6892,2.6549,2.6945,
 2.6776,2.6541,2.6521,2.6482,2.6117,2.6216,2.6215,2.5954,2.6052,2.5878,
 2.6137,2.6332,2.6033,2.6282,2.565 ,2.605 ,2.6015,2.58  ,2.5949,2.6356,
 2.5963,2.6226,2.5824,2.6132,2.5715,2.615 ,2.5805,2.5631,2.5268,2.563 ,
 2.5682,2.5571,2.5537,2.5602,2.5633,2.5564,2.5493,2.5405,2.532 ,2.5628,
 2.5652,2.5913,2.5627,2.5629,2.5474,2.5489,2.5483,2.5387,2.5654,2.6174,
 2.5919,2.6111,2.6227,2.6009,2.6687,2.6362,2.6328,2.622 ,2.5851,2.6312,
 2.5966,2.641 ,2.5791,2.5836,2.6266,2.6333,2.5795,2.6778,2.5761,2.6151,
 2.6028,2.6319,2.5922,2.5842,2.5518,2.5691,2.5732,2.5128,2.6036,2.6669])

build_time_2 = np.array([7.0142,5.6414,5.0208,4.7196,4.5032,4.38  ,4.1634,4.0799,4.1119,3.9689,
 4.0307,3.9428,3.8241,3.7994,3.7482,3.6335,3.6462,3.6303,3.6475,3.6133,
 3.5646,3.5343,3.504 ,3.5244,3.533 ,3.4779,3.4734,3.3898,3.4158,3.3826,
 3.4346,3.3878,3.3702,3.3495,3.3598,3.3226,3.314 ,3.3106,3.2581,3.248 ,
 3.2806,3.1961,3.2426,3.2412,3.1665,3.2168,3.1768,3.2118,3.2169,3.1375,
 3.1495,3.1121,3.155 ,3.1303,3.0953,3.0607,3.0656,3.0517,3.0966,3.0396,
 3.1025,3.0343,3.0688,3.0093,3.0178,2.9699,3.0114,3.0842,3.0641,3.0674,
 2.9907,2.9603,3.0457,2.9807,2.9517,2.9744,3.0066,2.9378,2.9923,2.9884,
 2.9527,3.    ,2.971 ,2.9132,2.9191,2.9108,2.9527,2.9037,2.937 ,2.8919,
 2.8759,2.8881,2.8754,2.8592,2.8681,2.8649,2.8543,2.8739,2.8115,2.8222,
 2.851 ,2.8344,2.816 ,2.8521,2.8171,2.8806,2.8783,2.7794,2.8205,2.7692,
 2.8026,2.8006,2.7953,2.8115,2.7983,2.8256,2.7508,2.7538,2.7658,2.7324,
 2.8298,2.7954,2.7537,2.7404,2.7414,2.7642,2.8061,2.7605,2.7995,2.7359,
 2.7381,2.7124,2.7223,2.7125,2.7274,2.7523,2.6788,2.6961,2.7008,2.71  ,
 2.6987,2.771 ,2.7331,2.7369,2.7134,2.7848,2.6661,2.7019,2.7332,2.6742,
 2.6536,2.6267,2.7366,2.7099,2.7415,2.7788,2.7407,2.7019,2.6987,2.7545,
 2.7413,2.6779,2.6879,2.7291,2.6927,2.6798,2.5968,2.6188,2.6282,2.6429,
 2.613 ,2.6431,2.6432,2.6415,2.5895,2.6567,2.6129,2.6357,2.6112,2.6593,
 2.6201,2.6124,2.6256,2.6288,2.5653,2.6043,2.6385,2.6119,2.5607,2.577 ,
 2.6102,2.6534,2.639 ,2.5832,2.5596,2.5973,2.5521,2.5513,2.5569,2.5415])

avg_build_times = (build_time_1 + build_time_2) / 2

print(np.array2string(avg_build_times, separator=', '))

[7.1882 , 5.61875, 4.99755, 4.71655, 4.5491 , 4.39755, 4.23215, 4.1526 ,
 4.0863 , 3.99325, 3.9872 , 3.90115, 3.8111 , 3.7827 , 3.7443 , 3.6662 ,
 3.6656 , 3.62565, 3.67055, 3.60855, 3.5812 , 3.5199 , 3.5051 , 3.49335,
 3.50235, 3.4717 , 3.449  , 3.4103 , 3.41755, 3.3853 , 3.3926 , 3.38315,
 3.3576 , 3.35025, 3.34725, 3.35325, 3.32135, 3.28755, 3.26075, 3.2443 ,
 3.27295, 3.22435, 3.2206 , 3.2188 , 3.1695 , 3.2072 , 3.15525, 3.16485,
 3.1598 , 3.12285, 3.13595, 3.09205, 3.1224 , 3.1138 , 3.1102 , 3.0875 ,
 3.08485, 3.0637 , 3.0837 , 3.07825, 3.099  , 3.08865, 3.064  , 3.03045,
 3.04775, 3.0012 , 3.00735, 3.0329 , 3.0346 , 3.04875, 3.02755, 2.9592 ,
 2.99545, 2.9619 , 2.9401 , 2.9578 , 2.9883 , 2.9365 , 2.95265, 2.95895,
 2.9937 , 2.97095, 2.9397 , 2.92245, 2.9601 , 2.9051 , 2.8952 , 2.89085,
 2.87795, 2.8316 , 2.845  , 2.86495, 2.8748 , 2.8629 , 2.85305, 2.8358 ,
 2.83115, 2.83865, 2.821  , 2.7942 , 2.80695, 2.8126 , 2.8159 , 2.811  ,
 2.78475, 2.8162 , 2.8123 , 2.74365, 2.78095, 2.752

In [7]:
search_time_1 = np.array([ 8.0974, 7.7395, 7.5807, 7.4125, 7.2498, 7.2097, 7.2031, 7.2478, 7.0801,
  7.0386, 6.9704, 7.1134, 6.9512, 7.0403, 7.1692, 7.0619, 7.136 , 7.2856,
  7.1485, 7.3481, 7.1292, 7.2695, 7.3943, 7.3844, 7.4814, 7.5287, 7.5602,
  7.6169, 7.5869, 7.6767, 7.7404, 7.8313, 7.9286, 7.8372, 7.9774, 8.1203,
  8.0693, 8.2152, 8.1739, 8.3238, 8.3143, 8.3848, 8.4837, 8.4677, 8.5997,
  8.7126, 8.7178, 8.7506, 8.7468, 8.8948, 8.9387, 9.0896, 9.118 , 9.3546,
  9.3951, 9.5973, 9.4982, 9.4976, 9.5799, 9.6599, 9.8806, 9.8947, 9.8619,
  9.9495,10.0875,10.0461, 9.946 ,10.1107,10.1615,10.4   ,10.3201,10.4116,
 10.3655,10.4104,10.5497,10.5873,10.6058,10.8269,10.9909,11.0352,11.1031,
 11.1382,11.1537,11.2419,11.3087,11.2211,11.207 ,11.3268,11.3532,11.44  ,
 11.6238,11.7091,11.8108,11.9248,11.9437,11.9772,12.0418,11.863 ,11.9014,
 12.1444,12.1385,12.0843,12.0992,12.1406,12.1396,12.0547,12.1637,12.1284,
 12.1559,12.1362,12.1703,12.31  ,12.3116,12.5729,12.5028,12.5805,12.6329,
 12.6015,12.7366,12.8113,12.9205,13.0218,13.0834,12.9948,13.2719,13.2508,
 13.5633,13.4759,13.6239,13.4838,13.6313,13.6302,13.8227,13.62  ,13.6528,
 13.5197,13.6154,13.6397,13.7002,13.7622,13.9422,13.9075,13.7421,13.973 ,
 13.9685,14.0084,13.9163,14.1464,14.1242,14.2869,14.3171,14.4949,14.3849,
 14.4655,14.4116,14.6257,14.4958,14.5562,14.4369,14.5479,14.5238,14.6911,
 15.2591,15.1383,15.3113,15.4556,15.4375,15.6287,15.9713,16.1837,16.3948,
 16.4118,16.5663,16.6005,16.7605,16.8865,16.7261,16.7645,16.8431,16.8349,
 17.1658,17.0991,16.7398,16.8266,17.1106,17.0456,16.9212,17.1719,17.0314,
 17.0264,17.2378,17.6386,17.3723,17.389 ,17.2023,17.3418,17.4241,17.4657,
 17.7703,18.0178])

search_time_2 = np.array([ 8.1423, 7.8563, 7.5471, 7.3902, 7.2619, 7.1546, 7.1466, 7.2408, 7.1695,
  7.1346, 7.1471, 7.1442, 7.1256, 7.1854, 6.9544, 7.0917, 7.1587, 7.1684,
  7.2324, 7.2791, 7.3372, 7.2604, 7.3655, 7.4757, 7.659 , 7.6535, 7.6497,
  7.5568, 7.7421, 7.8451, 7.8018, 7.949 , 7.9304, 8.1323, 8.134 , 8.1599,
  8.2107, 8.262 , 8.4147, 8.3111, 8.4649, 8.475 , 8.5553, 8.5126, 8.6693,
  8.7861, 8.8254, 8.8882, 8.9751, 8.8297, 9.12  , 9.0503, 9.2299, 9.3156,
  9.2931, 9.3524, 9.4994, 9.6231, 9.6714, 9.5127, 9.4739, 9.9696, 9.8608,
  9.9178,10.0483,10.1523, 9.9519,10.0994,10.4492,10.3276,10.654 ,10.4578,
 10.3623,10.4966,10.5626,10.7107,10.8448,10.9509,11.0726,11.1408,11.3318,
 11.2599,11.2221,11.1998,11.3474,11.4275,11.4669,11.6342,11.552 ,11.6479,
 11.799 ,11.8435,11.8473,11.9803,12.0973,12.2061,12.287 ,12.096 ,12.3937,
 12.453 ,12.7038,12.4053,12.4155,12.8935,12.8816,13.0539,12.8931,12.6987,
 12.7134,12.8485,12.866 ,12.9319,12.8357,13.2379,13.0956,13.1041,13.1316,
 13.292 ,13.3582,13.5801,13.8974,13.6026,13.6478,13.7969,14.253 ,14.3452,
 14.5931,14.622 ,14.5777,14.613 ,14.4168,14.4646,14.5139,14.5021,14.5219,
 14.3335,14.4088,14.5484,14.5532,14.6467,14.7443,14.8045,14.8622,14.9229,
 15.0158,14.9331,14.8556,15.2333,15.1216,15.2165,15.0823,15.5776,15.4343,
 15.8399,16.1019,16.1084,15.9846,15.7779,16.2399,15.9229,16.0471,16.2333,
 15.8989,16.629 ,16.4941,16.2382,16.1205,16.2853,16.2027,16.3965,16.3844,
 16.2057,16.5463,16.5008,16.4077,16.6767,16.9904,16.8486,16.9594,16.9222,
 17.006 ,17.0533,16.9267,17.0747,17.09  ,17.3607,16.9268,16.9973,17.1563,
 17.2721,17.6104,17.8351,17.928 ,17.4099,17.3764,17.3642,17.4682,17.4948,
 17.6417,17.6048])

avg_search_times = (search_time_1 + search_time_2) / 2

print(np.array2string(avg_search_times, separator=', '))

[ 8.11985,  7.7979 ,  7.5639 ,  7.40135,  7.25585,  7.18215,  7.17485,
  7.2443 ,  7.1248 ,  7.0866 ,  7.05875,  7.1288 ,  7.0384 ,  7.11285,
  7.0618 ,  7.0768 ,  7.14735,  7.227  ,  7.19045,  7.3136 ,  7.2332 ,
  7.26495,  7.3799 ,  7.43005,  7.5702 ,  7.5911 ,  7.60495,  7.58685,
  7.6645 ,  7.7609 ,  7.7711 ,  7.89015,  7.9295 ,  7.98475,  8.0557 ,
  8.1401 ,  8.14   ,  8.2386 ,  8.2943 ,  8.31745,  8.3896 ,  8.4299 ,
  8.5195 ,  8.49015,  8.6345 ,  8.74935,  8.7716 ,  8.8194 ,  8.86095,
  8.86225,  9.02935,  9.06995,  9.17395,  9.3351 ,  9.3441 ,  9.47485,
  9.4988 ,  9.56035,  9.62565,  9.5863 ,  9.67725,  9.93215,  9.86135,
  9.93365, 10.0679 , 10.0992 ,  9.94895, 10.10505, 10.30535, 10.3638 ,
 10.48705, 10.4347 , 10.3639 , 10.4535 , 10.55615, 10.649  , 10.7253 ,
 10.8889 , 11.03175, 11.088  , 11.21745, 11.19905, 11.1879 , 11.22085,
 11.32805, 11.3243 , 11.33695, 11.4805 , 11.4526 , 11.54395, 11.7114 ,
 11.7763 , 11.82905, 11.95255, 12.0205 , 12.09165, 12.1644 , 11.9795 ,
 12.14